## Problem 3
https://projecteuler.net/problem=3

In [1]:
using Primes, BenchmarkTools
include("./euler.jl");

In [2]:
maximum(keys(Primes.factor(600851475143)))

6857

In [3]:
b = @benchmark maximum(keys(Primes.factor(600851475143)))

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.708 μs …  16.913 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.833 μs ± 603.946 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    █▄                                                         
  ▂███▅▂▃▄▃▂▂▂▂▂▂▂▂▃▆▆▄▂▂▂▂▁▁▂▁▁▁▂▁▁▁▂▂▁▂▂▂▁▂▂▂▂▁▁▁▂▂▁▂▁▂▁▁▂▂ ▂
  1.71 μs         Histogram: frequency by time        2.44 μs <

 Memory estimate: 128 bytes, allocs estimate: 2.

In [4]:
euler.save_benchmark(3, b);